In [1]:

# # Web scraping stat BRK

In [2]:


import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [3]:


url = "https://leagueoflegends.fandom.com/wiki/Blade_of_the_Ruined_King"
url

'https://leagueoflegends.fandom.com/wiki/Blade_of_the_Ruined_King'

In [4]:


response = requests.get(url)
html = BeautifulSoup(response.text, "html.parser")


# ## Bases stats

In [5]:


brk_stat = html.findAll("div",{"class" : "pi-data-value pi-font"})[:3]
brk_stat

[<div class="pi-data-value pi-font">+40 <a href="/wiki/Attack_damage" title="Attack damage">attack damage</a></div>,
 <div class="pi-data-value pi-font">+25% <a href="/wiki/Attack_speed" title="Attack speed">attack speed</a></div>,
 <div class="pi-data-value pi-font">+8% <a href="/wiki/Life_steal" title="Life steal">life steal</a></div>]

In [6]:


for idx,stats in enumerate(brk_stat):
    brk_stat[idx] = re.search(r'(\d+)', stats.text).group(1)

# take account of %
brk_stat = list(map(int,brk_stat))
brk_stat[1] /=100
brk_stat[2] /=100
brk_stat


# ## HP % Passive

[40, 0.25, 0.08]


In [7]:


#brk_passives = html.findAll("div",{"class" : "pi-data-value pi-font"})[3:5]
brk_passive  = html.find("span",{"style": "white-space:pre; position:relative;", "data-game"  : "lol"})
brk_passive = brk_passive.text
brk_passive = re.search(r'(\d+)', brk_passive).group(1)
brk_passive = float(brk_passive) / 100
brk_passive

0.12

In [8]:


brk_passive_range  = html.find("span",{"style": "white-space:pre; position:relative;", "data-game"  : "lol" , "data-tip" : "Ranged"})
brk_passive_range = brk_passive_range.text
brk_passive_range = re.search(r'(\d+)', brk_passive_range).group(1)
brk_passive_range = float(brk_passive_range) / 100
brk_passive_range


# ## Slow passive

0.09

In [9]:


brk_3autos = html.find("span",{"style" : "position:relative; border-bottom:1px dotted; cursor:help;"})


40;47;54;61;68;75;82;89;96;103
1;10;11;12;13;14;15;16;17;18


In [10]:


three_autos_lvl = brk_3autos['data-top_values'].split(";")

three_autos_dmg = brk_3autos['data-bot_values'].split(";")


three_autos_dict = {key: value for key, value in zip(three_autos_lvl, three_autos_dmg)}
three_autos_dict


# ## Item creation

['1', '10', '11', '12', '13', '14', '15', '16', '17', '18']
['40', '47', '54', '61', '68', '75', '82', '89', '96', '103']
{'1': '40', '10': '47', '11': '54', '12': '61', '13': '68', '14': '75', '15': '82', '16': '89', '17': '96', '18': '103'}


In [11]:

if __name__ == "__main__":
    import sys
    sys.path.append("..")
    from class_item import Item
else :
    from class_item import Item
import copy

In [12]:


brk_dict = {"ad" : brk_stat[0], "as" : brk_stat[1], "lifesteal" : brk_stat[2]}
brk_dict['passive'] = brk_passive
#brk_dict['three_autos'] = three_autos_dict
brk_dict_range = copy.deepcopy(brk_dict)
brk_dict_range['passive'] = brk_passive_range

In [13]:


BRK_melee = Item("brk", brk_dict)
BRK_range = Item("brk", brk_dict_range)

In [15]:
def createBRK(melee : bool = True):
    if melee:
        return Item("brk", brk_dict)
    else:
        return Item("brk", brk_dict_range)